# Introducción a Dask
José Fernando Zea

En npartitions se muestra en cuantas particiones es dividido el dataframne. Debido a que el archvivo tiene 9 GB de tamaño se lleva a cabo 163 particiones de 2 Gigas cada una.Esto significa que en lugar de colocar todo el archivo en RAM, se procesará cada vez un archivo de 64 MB.

In [1]:
import os
os.chdir(r'C:\Users\Stats\Documents\Bigdata_externado\sesión 4\datos')


In [2]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pyarrow

#df = dd.read_csv('nyc-parking-tickets/*2017.csv')
df = dd.read_csv('2018_Yellow_Taxi_Trip_Data.csv')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
npartitions=163,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,int64,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
df.index.size

dd.Scalar<size-ag..., dtype=int32>

El número de filas de la tabla como ya se sabe de ejercicios anteriores es de 112234626

In [3]:
df.index.size.compute()

112234626

Si se desesa calcular una operación de cuantificar el número de faltantes se realiza una operación que aprovecha cada uno de los procesarores en 409 tareas, Dask cuando ejecuta una sentencia muestra la información en metadata, y realiza evaluación perezosa (lazy evaluation), solo muiestra los resultados hasta que explicitamente se le dice (analogía receta de concina)

In [5]:
missing_values = df.isnull().sum()
missing_values

Dask Series Structure:
npartitions=1
DOLocationID     int64
trip_distance      ...
dtype: int64
Dask Name: dataframe-sum-agg, 490 tasks

In [9]:
missing_count = ((missing_values / df.index.size) * 100)
missing_count

Dask Series Structure:
npartitions=1
DOLocationID     float64
trip_distance        ...
dtype: float64
Dask Name: mul, 819 tasks

In [10]:
with ProgressBar():
    missing_count_pct = missing_count.compute()
missing_count_pct

[########################################] | 100% Completed |  2min 38.1s


VendorID                 0.0
tpep_pickup_datetime     0.0
tpep_dropoff_datetime    0.0
passenger_count          0.0
trip_distance            0.0
RatecodeID               0.0
store_and_fwd_flag       0.0
PULocationID             0.0
DOLocationID             0.0
payment_type             0.0
fare_amount              0.0
extra                    0.0
mta_tax                  0.0
tip_amount               0.0
tolls_amount             0.0
improvement_surcharge    0.0
total_amount             0.0
dtype: float64

Observe que cuando se recolecta la información el objeto es una serie de pandas que se puede manejar de la manera usual, es conveniente conservar las variables que tengan menos del 20% de faltantes. Identificamos cuales son las columnas que vamos a conservar, con persist guardamos en disco sólo las columnas que nos interesan

In [ ]:
columns_to_drop = missing_count_pct[missing_count_pct > 20].index
with ProgressBar():
    df_dropped = df.drop(columns_to_drop, axis=1).persist()

# Estructura


In [14]:
person_IDs = [1,2,3]
person_last_names = ['Smith', 'Williams', 'Williams']
person_first_names = ['John', 'Bill', 'Jane']
person_DOBs = ['1982-10-06', '1990-07-04', '1989-05-06']

In [15]:
import pandas as pd
import dask.dataframe as dd

# Creating all the data as lists
person_IDs = [1,2,3,4,5,6,7,8,9,10]
person_last_names = ['Smith', 'Williams', 'Williams','Jackson','Johnson','Smith','Anderson','Christiansen','Carter','Davidson']
person_first_names = ['John', 'Bill', 'Jane','Cathy','Stuart','James','Felicity','Liam','Nancy','Christina']
person_DOBs = ['1982-10-06', '1990-07-04', '1989-05-06', '1974-01-24', '1995-06-05', '1984-04-16', '1976-09-15', '1992-10-02', '1986-02-05', '1993-08-11']

# Storing the data in a Pandas DataFrame
people_pandas_df = pd.DataFrame({'Person ID': person_IDs, 
              'Last Name': person_last_names, 
              'First Name': person_first_names,
             'Date of Birth': person_DOBs},
            columns=['Person ID', 'Last Name', 'First Name', 'Date of Birth'])

# Converting the Pandas DataFrame to a Dask DataFrame
people_dask_df = dd.from_pandas(people_pandas_df, npartitions=2)

Es posible para un Dask Dataframe definir el número de particiones explictamente, en este caso 2, normalmente si el conjunto de datos es muy pequeño Dask particiona en un solo archivo.

Tarea: revisar una operación en el conjunto de datos de empresas (que es pequeño) como es más eficiente la operación sin en Dask o en  pandas

Puede revisarse los cortes de cada partición, la partición 1 va del índice 0 al 5 (sin incluir al 5), la dos va del 5 al 9 (incluyendo al 9 puesto que es la última partición.)

In [16]:
print(people_dask_df.divisions)
print(people_dask_df.npartitions) # Partición 1 va de 0 a 4 (no incluye 5), la otra de 5 a 9 (incluye el valor 9)

(0, 5, 9)
2


El número de filas de cada partición puede calcularse con la función map_partitions que cuantifica para cada partición una operación por fila.

In [17]:
people_dask_df.map_partitions(lambda x: len(x)).compute(

0    5
1    5
dtype: int64

# Leer de un archivo parquet

In [20]:
os.chdir(r'C:\Users\Stats\Documents\Bigdata_externado\sesión 4\datos')

#df = dd.read_csv('nyc-parking-tickets/*2017.csv')
data = dd.read_parquet('parquet')
data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
npartitions=113,,,,,,,,,,,,,,,,,
,float64,object,object,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [21]:
data.index.size.compute()

112234626

In [ ]:
# Filtrando y seleccionadno variables

In [22]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1.0,09/11/2018 03:14:12 PM,09/11/2018 03:39:30 PM,2.0,10.10,1.0,N,138.0,225.0,1.0,30.0,0.0,0.5,6.15,0.0,0.3,36.95
1,2.0,09/11/2018 03:13:47 PM,09/11/2018 03:25:04 PM,1.0,1.11,1.0,N,90.0,113.0,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30
2,2.0,09/11/2018 03:29:21 PM,09/11/2018 03:46:17 PM,1.0,2.13,1.0,N,113.0,162.0,1.0,12.5,0.0,0.5,2.66,0.0,0.3,15.96
3,1.0,09/11/2018 03:25:58 PM,09/11/2018 03:50:44 PM,2.0,2.50,1.0,N,239.0,161.0,2.0,16.0,0.0,0.5,0.00,0.0,0.3,16.80
4,1.0,09/11/2018 03:52:57 PM,09/11/2018 03:58:05 PM,2.0,0.60,1.0,N,161.0,161.0,1.0,5.0,0.0,0.5,1.15,0.0,0.3,6.95


Seleccionaremos las variables identificador de viaje, 

In [39]:
with ProgressBar():
    display(data['RatecodeID'].head())

[########################################] | 100% Completed |  0.1s


0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: RatecodeID, dtype: float64

Si adicional a lo anterior queremos seleccionar varias columnas, por ejemplo el ID, el tipo de tarifa y la cantidad total

In [40]:
with ProgressBar():
    display(data[['VendorID', 'RatecodeID', 'total_amount']].head())

[########################################] | 100% Completed |  0.1s


,VendorID,RatecodeID,total_amount
0,1.0,1.0,36.95
1,2.0,1.0,10.30
2,2.0,1.0,15.96
3,1.0,1.0,16.80
4,1.0,1.0,6.95


Si se quisiera eliminar algunas que no se desearan tener, por ejemplo la fecha de de inicio y fin de carrera:

In [42]:
with ProgressBar():
    display(data.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1).head())

[########################################] | 100% Completed |  1.2s


,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1.0,2.0,10.10,1.0,N,138.0,225.0,1.0,30.0,0.0,0.5,6.15,0.0,0.3,36.95
1,2.0,1.0,1.11,1.0,N,90.0,113.0,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30
2,2.0,1.0,2.13,1.0,N,113.0,162.0,1.0,12.5,0.0,0.5,2.66,0.0,0.3,15.96
3,1.0,2.0,2.50,1.0,N,239.0,161.0,2.0,16.0,0.0,0.5,0.00,0.0,0.3,16.80
4,1.0,2.0,0.60,1.0,N,161.0,161.0,1.0,5.0,0.0,0.5,1.15,0.0,0.3,6.95


#  Renombrar variables

In [45]:
# 'tpep_pickup_datetime', 'tpep_dropoff_datetime'
data = data.rename(columns={'tpep_pickup_datetime':'fechahora_inicioCarrera', 'tpep_dropoff_datetime':'fechahora_finCarrera'})
data.head()

,VendorID,fechahora_inicioCarrera,fechahora_finCarrera,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1.0,09/11/2018 03:14:12 PM,09/11/2018 03:39:30 PM,2.0,10.10,1.0,N,138.0,225.0,1.0,30.0,0.0,0.5,6.15,0.0,0.3,36.95
1,2.0,09/11/2018 03:13:47 PM,09/11/2018 03:25:04 PM,1.0,1.11,1.0,N,90.0,113.0,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30
2,2.0,09/11/2018 03:29:21 PM,09/11/2018 03:46:17 PM,1.0,2.13,1.0,N,113.0,162.0,1.0,12.5,0.0,0.5,2.66,0.0,0.3,15.96
3,1.0,09/11/2018 03:25:58 PM,09/11/2018 03:50:44 PM,2.0,2.50,1.0,N,239.0,161.0,2.0,16.0,0.0,0.5,0.00,0.0,0.3,16.80
4,1.0,09/11/2018 03:52:57 PM,09/11/2018 03:58:05 PM,2.0,0.60,1.0,N,161.0,161.0,1.0,5.0,0.0,0.5,1.15,0.0,0.3,6.95


# Selección de filas

In [49]:
with ProgressBar():
    display(data.loc[3:5].head())

[########################################] | 100% Completed |  1.0s


C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\core.py:6194: UserWarning: Insufficient elements for `head`. 5 elements requested, only 3 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,VendorID,fechahora_inicioCarrera,fechahora_finCarrera,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
3,1.0,09/11/2018 03:25:58 PM,09/11/2018 03:50:44 PM,2.0,2.50,1.0,N,239.0,161.0,2.0,16.0,0.0,0.5,0.00,0.0,0.3,16.80
4,1.0,09/11/2018 03:52:57 PM,09/11/2018 03:58:05 PM,2.0,0.60,1.0,N,161.0,161.0,1.0,5.0,0.0,0.5,1.15,0.0,0.3,6.95
5,2.0,09/11/2018 03:04:48 PM,09/11/2018 03:13:50 PM,1.0,1.64,1.0,N,234.0,161.0,1.0,8.0,0.0,0.5,2.20,0.0,0.3,11.00


# Recodificación

Las tarifas a aeropuertos se unifican, las demás se conservan

In [64]:
condition_airport = data['RatecodeID'].isin([2, 3, 4])
tipo_tarifa = data['RatecodeID'].mask(condition_airport, 'Aeropuerto')
data2 = data.assign(RatecodeID=tipo_tarifa)
data2.head()
# Listing 5.21
with ProgressBar():
    display(data2['RatecodeID'].value_counts().compute())

[########################################] | 100% Completed |  2min 12.8s


1.0           108895988
Aeropuerto      2888620
5.0              443848
99.0               5222
6.0                 948
Name: RatecodeID, dtype: int64

# Filtros

Seleccionar las carreras de menos de una milla de tarifa corriente (RateCodeID = 1)

In [67]:
data.head()

,VendorID,fechahora_inicioCarrera,fechahora_finCarrera,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1.0,09/11/2018 03:14:12 PM,09/11/2018 03:39:30 PM,2.0,10.10,1.0,N,138.0,225.0,1.0,30.0,0.0,0.5,6.15,0.0,0.3,36.95
1,2.0,09/11/2018 03:13:47 PM,09/11/2018 03:25:04 PM,1.0,1.11,1.0,N,90.0,113.0,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30
2,2.0,09/11/2018 03:29:21 PM,09/11/2018 03:46:17 PM,1.0,2.13,1.0,N,113.0,162.0,1.0,12.5,0.0,0.5,2.66,0.0,0.3,15.96
3,1.0,09/11/2018 03:25:58 PM,09/11/2018 03:50:44 PM,2.0,2.50,1.0,N,239.0,161.0,2.0,16.0,0.0,0.5,0.00,0.0,0.3,16.80
4,1.0,09/11/2018 03:52:57 PM,09/11/2018 03:58:05 PM,2.0,0.60,1.0,N,161.0,161.0,1.0,5.0,0.0,0.5,1.15,0.0,0.3,6.95


In [73]:
# Listing 6.3
with ProgressBar():
    condition = (data['RatecodeID'] == 1) & (data['trip_distance'] <= 1)
    consulta = data[condition]
consulta.head()

,VendorID,fechahora_inicioCarrera,fechahora_finCarrera,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
4,1.0,09/11/2018 03:52:57 PM,09/11/2018 03:58:05 PM,2.0,0.6,1.0,N,161.0,161.0,1.0,5.0,0.0,0.5,1.15,0.0,0.3,6.95
8,1.0,09/11/2018 03:34:02 PM,09/11/2018 03:44:07 PM,1.0,0.8,1.0,N,237.0,161.0,1.0,7.5,0.0,0.5,1.65,0.0,0.3,9.95
10,1.0,09/11/2018 03:11:05 PM,09/11/2018 03:14:56 PM,1.0,0.7,1.0,N,41.0,24.0,2.0,5.0,0.0,0.5,0.00,0.0,0.3,5.80
12,1.0,09/11/2018 03:03:47 PM,09/11/2018 03:09:58 PM,1.0,0.6,1.0,N,170.0,100.0,1.0,5.5,0.0,0.5,1.25,0.0,0.3,7.55
14,1.0,09/11/2018 03:39:37 PM,09/11/2018 03:42:41 PM,1.0,0.6,1.0,N,237.0,236.0,1.0,4.5,0.0,0.5,1.06,0.0,0.3,6.36


In [75]:
consulta.index.size.compute()

31043945

# Algunas estadísticas

In [77]:
from dask.array import stats as dask_stats
with ProgressBar():
    mean = data['trip_distance'].mean().compute()
    stdev = data['trip_distance'].std().compute()
    minimum = data['trip_distance'].min().compute()
    maximum = data['trip_distance'].max().compute()
    skewness = float(dask_stats.skew(data['trip_distance'].values).compute())

[########################################] | 100% Completed |  1min 40.8s
[########################################] | 100% Completed |  1min 47.8s
[########################################] | 100% Completed |  1min 46.3s
[########################################] | 100% Completed |  1min 47.0s
[########################################] | 100% Completed |  1min 47.6s


In [79]:
mean, stdev, minimum, maximum, skewness

(2.9294395068416765, 18.33739452089531, 0.0, 189483.84, 9831.861648198754)

Analicemos los tipos de pago

In [91]:
with ProgressBar():
    consulta2 = df.groupby('payment_type')['VendorID'].count().compute()
consulta2.head()                                            

[########################################] | 100% Completed |  2min 29.9s


payment_type
1    77928307
2    33556849
3      582599
4      166868
5           3
Name: VendorID, dtype: int64

In [92]:
consulta2.to_frame().sort_values(by=['payment_type'])


,VendorID
payment_type,
1,77928307
2,33556849
3,582599
4,166868
5,3


Calcular el promedio de la distancia, la tarifa agregada por tipo de pago

In [94]:
with ProgressBar():
    consulta3 = df.groupby('payment_type').agg({'trip_distance': 'mean', 'total_amount': 'mean'})
    consulta3.compute()

[########################################] | 100% Completed |  2min 31.5s


In [96]:
consulta3.head()

,trip_distance,total_amount
payment_type,,
1,3.061414,17.664706
2,2.626821,13.352206
3,2.725489,19.045940
4,2.864860,12.742401
5,4.166667,32.513333
